In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import root_mean_squared_error

In [2]:
import mlflow
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2025/09/09 11:43:19 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/09/09 11:43:19 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/workspaces/mlops/02-experiment-tracking/mlruns/1', creation_time=1757415811725, experiment_id='1', last_update_time=1757415811725, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [3]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']
target = 'duration'
def get_df(parquet_file_path):
    df = pd.read_parquet(parquet_file_path)
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df["duration"] = df["duration"].dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <=60)]
    df[categorical] = df[categorical].astype(str)
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']   # creating a new feature
    return df

In [4]:
train_df = get_df("data/yellow_tripdata_2023-01.parquet")
val_df = get_df("data/yellow_tripdata_2023-02.parquet")
train_df = train_df.head(60000)
val_df = val_df.head(50000)

In [5]:
dv = DictVectorizer()
new_feature = ['PU_DO']
train_dict = train_df[new_feature + numerical].to_dict(orient='records')
val_dict = val_df[new_feature + numerical].to_dict(orient='records')

x_train = dv.fit_transform(train_dict)
y_train = train_df[target].values

x_val = dv.transform(val_dict)
y_val = val_df[target].values

In [6]:
with mlflow.start_run():
    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    mlflow.set_tag('model', 'Ridge')
    model = Ridge(alpha=alpha, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)
    print(f"RMSE: {rmse}")
    with open('model.bin','wb') as f_out:
        pickle.dump((dv,model),f_out)
    mlflow.log_artifact('model.bin')

RMSE: 6.32570387422938


In [6]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll.base import scope

In [7]:
dtrain= xgb.DMatrix(x_train, label=y_train)
dvalid = xgb.DMatrix(x_val, label=y_val)

In [9]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag('model','XGBoost')
        mlflow.log_params(params)
        booster = xgb.train(params=params, dtrain=dtrain, num_boost_round=10, 
                            evals=[(dvalid, 'validation')], early_stopping_rounds=5)
        y_pred = booster.predict(dvalid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)
    return {'loss': rmse, 'status': STATUS_OK}

In [10]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 10, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

In [11]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=4,
    trials=Trials(),
    verbose=False
)

[0]	validation-rmse:6.68610


/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [11:35:25] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[1]	validation-rmse:6.18857
[2]	validation-rmse:6.11031
[3]	validation-rmse:6.08570
[4]	validation-rmse:6.08079
[5]	validation-rmse:6.07901
[6]	validation-rmse:6.07476
[7]	validation-rmse:6.07358
[8]	validation-rmse:6.06610
[9]	validation-rmse:6.06371
[0]	validation-rmse:7.30666
[1]	validation-rmse:6.44034
[2]	validation-rmse:6.19773
[3]	validation-rmse:6.11534


/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [11:35:26] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[4]	validation-rmse:6.09984
[5]	validation-rmse:6.08296
[6]	validation-rmse:6.08120
[7]	validation-rmse:6.07333
[8]	validation-rmse:6.07220
[9]	validation-rmse:6.07829
[0]	validation-rmse:9.51564
[1]	validation-rmse:8.99555


/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [11:35:27] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[2]	validation-rmse:8.54806
[3]	validation-rmse:8.16425
[4]	validation-rmse:7.83436
[5]	validation-rmse:7.55418
[6]	validation-rmse:7.31681
[7]	validation-rmse:7.11442
[8]	validation-rmse:6.94322
[9]	validation-rmse:6.79849
[0]	validation-rmse:9.46042
[1]	validation-rmse:8.90098
[2]	validation-rmse:8.42710
[3]	validation-rmse:8.02810


/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [11:35:28] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[4]	validation-rmse:7.69237
[5]	validation-rmse:7.41454
[6]	validation-rmse:7.17991
[7]	validation-rmse:6.98866
[8]	validation-rmse:6.83110
[9]	validation-rmse:6.69917


In [8]:
best_params = {
    "learning_rate": 0.6856143053942376,
    "max_depth": 7,
    "min_child_weight": 4.434221562748838,
    "objective": "reg:linear",
    "reg_alpha": 0.16562147689397697,
    "reg_lambda": 0.06268756235943868,
    "seed": 42
}
with mlflow.start_run():
    mlflow.log_params(best_params)
    booster = xgb.train(params=best_params, dtrain=dtrain, num_boost_round=10, 
                                evals=[(dvalid, 'validation')], early_stopping_rounds=5)
    y_pred = booster.predict(dvalid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse',rmse)
    mlflow.xgboost.log_model(booster,artifact_path='xgb_model')

[0]	validation-rmse:6.68610
[1]	validation-rmse:6.18857


/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/xgboost/callback.py:386: UserWarning: [11:43:52] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[2]	validation-rmse:6.11031
[3]	validation-rmse:6.08570
[4]	validation-rmse:6.08079
[5]	validation-rmse:6.07901
[6]	validation-rmse:6.07476
[7]	validation-rmse:6.07358
[8]	validation-rmse:6.06610
[9]	validation-rmse:6.06371


2025/09/09 11:43:53 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
/home/codespace/miniconda/envs/mlops-env/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [11:43:53] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  xgb_model.save_model(model_data_path)


: 